In [1]:
import os
import numpy as np 
from glob import glob
from PIL import Image
import cv2
import slideio
import json
from skimage.draw import polygon2mask
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")


In [2]:
in_situ_whi_list=glob('../../data/1-cycle_30%_중간데이터/1_insitu/WSI/*.tiff')
normal_whi_list=glob('../../data/1-cycle_30%_중간데이터/0_normal/WSI/*.tiff')
malignant_whi_list=glob('../../data/1-cycle_30%_중간데이터/2_malignant/WSI/*.tiff')
image_path='../../data/1-cycle_30%_중간데이터/segmentation/train/image/'
normal_mask_path='../../data/1-cycle_30%_중간데이터/segmentation/train/polygon/NT_normal/'
tumor_mask_path='../../data/1-cycle_30%_중간데이터/segmentation//train/polygon/TP_tumor/'
test_image_path='../../data/1-cycle_30%_중간데이터/segmentation/test/image/'
test_normal_mask_path='../../data/1-cycle_30%_중간데이터/segmentation/test/polygon/NT_normal/'
test_tumor_mask_path='../../data/1-cycle_30%_중간데이터/segmentation//test/polygon/TP_tumor/'

def size_ratio(scene,img_size):
    width = scene.rect[2]
    height = scene.rect[3]
    ratio=0
    inverse_ratio=0
    img_width=0
    img_height=0
    if width>height:
        ratio=img_size/width
        inverse_ratio=width/img_size
        img_width=img_size
        img_height=height*ratio
    else:
        ratio=img_size/height
        inverse_ratio=height/img_size
        img_height=img_size
        img_width=width*ratio
        
    return int(img_width),int(img_height),inverse_ratio



In [3]:
json_list=[f.replace('/WSI/', '/json/') for f in in_situ_whi_list]
json_list=[f.replace('.tiff', '.json') for f in json_list]
for i in tqdm(range(len(in_situ_whi_list))):
    slide = slideio.open_slide(in_situ_whi_list[i], "GDAL")
    fileName=os.path.basename(os.path.splitext(in_situ_whi_list[i])[0])
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    tumor_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    normal_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    total_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    with open(json_list[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    image_shape=(img_height,img_width)
    for j in range(polygon_count):
        if json_object['files'][0]['objects'][j]['label']=='TP_tumor':
            polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])*1/ratio
            polygon1=np.copy(polygon)
            polygon1[:,0]=polygon[:,1]
            polygon1[:,1]=polygon[:,0]
            mask=polygon2mask(image_shape,polygon1)
            tumor_mask=mask+tumor_mask
    tumor_mask=np.where(tumor_mask>0,255,0)
    tumor_mask=cv2.cvtColor(tumor_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    t, t_otsu = cv2.threshold(image[:,:,1], -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    t_otsu = cv2.morphologyEx(t_otsu, cv2.MORPH_OPEN, k)
    total_mask=255-cv2.cvtColor(t_otsu.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    normal_mask=np.where((total_mask-tumor_mask)>200,255,0)
    if random.randrange(1,6)==5:
        cv2.imwrite(test_tumor_mask_path+fileName+'.tiff',tumor_mask )
        cv2.imwrite(test_normal_mask_path+fileName+'.tiff',normal_mask.astype(np.uint8) )
        cv2.imwrite(test_image_path+fileName+'.tiff',image )
    else:
        cv2.imwrite(tumor_mask_path+fileName+'.tiff',tumor_mask )
        cv2.imwrite(normal_mask_path+fileName+'.tiff',normal_mask.astype(np.uint8) )
        cv2.imwrite(image_path+fileName+'.tiff',image )

  0%|          | 0/1485 [00:00<?, ?it/s]

In [4]:
json_list=[f.replace('/WSI/', '/json/') for f in malignant_whi_list]
json_list=[f.replace('.tiff', '.json') for f in json_list]
for i in tqdm(range(len(malignant_whi_list))):
    slide = slideio.open_slide(malignant_whi_list[i], "GDAL")
    fileName=os.path.basename(os.path.splitext(malignant_whi_list[i])[0])
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    tumor_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    normal_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    total_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    with open(json_list[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    image_shape=(img_height,img_width)
    for j in range(polygon_count):
        if json_object['files'][0]['objects'][j]['label']=='TP_tumor':
            polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])*1/ratio
            polygon1=np.copy(polygon)
            polygon1[:,0]=polygon[:,1]
            polygon1[:,1]=polygon[:,0]
            mask=polygon2mask(image_shape,polygon1)
            tumor_mask=mask+tumor_mask
    tumor_mask=np.where(tumor_mask>0,255,0)
    tumor_mask=cv2.cvtColor(tumor_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    t, t_otsu = cv2.threshold(image[:,:,1], -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    t_otsu = cv2.morphologyEx(t_otsu, cv2.MORPH_OPEN, k)
    total_mask=255-cv2.cvtColor(t_otsu.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    normal_mask=np.where((total_mask-tumor_mask)>200,255,0)
    if random.randrange(1,6)==5:
        cv2.imwrite(test_tumor_mask_path+fileName+'.tiff',tumor_mask )
        cv2.imwrite(test_normal_mask_path+fileName+'.tiff',normal_mask.astype(np.uint8) )
        cv2.imwrite(test_image_path+fileName+'.tiff',image )
    else:
        cv2.imwrite(tumor_mask_path+fileName+'.tiff',tumor_mask )
        cv2.imwrite(normal_mask_path+fileName+'.tiff',normal_mask.astype(np.uint8) )
        cv2.imwrite(image_path+fileName+'.tiff',image )

  0%|          | 0/1765 [00:00<?, ?it/s]

In [5]:
json_list=[f.replace('/WSI/', '/json/') for f in normal_whi_list]
json_list=[f.replace('.tiff', '.json') for f in json_list]
for i in tqdm(range(len(normal_whi_list))):
    slide = slideio.open_slide(normal_whi_list[i], "GDAL")
    fileName=os.path.basename(os.path.splitext(normal_whi_list[i])[0])
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    tumor_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    normal_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    total_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    with open(json_list[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    image_shape=(img_height,img_width)
    for j in range(polygon_count):
        if json_object['files'][0]['objects'][j]['label']=='TP_tumor':
            polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])*1/ratio
            polygon1=np.copy(polygon)
            polygon1[:,0]=polygon[:,1]
            polygon1[:,1]=polygon[:,0]
            mask=polygon2mask(image_shape,polygon1)
            tumor_mask=mask+tumor_mask
    tumor_mask=np.where(tumor_mask>0,255,0)
    tumor_mask=cv2.cvtColor(tumor_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    t, t_otsu = cv2.threshold(image[:,:,1], -1, 255,  cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    t_otsu = cv2.morphologyEx(t_otsu, cv2.MORPH_OPEN, k)
    total_mask=255-cv2.cvtColor(t_otsu.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    normal_mask=np.where((total_mask-tumor_mask)>200,255,0)
    if random.randrange(1,6)==5:
        cv2.imwrite(test_tumor_mask_path+fileName+'.tiff',tumor_mask )
        cv2.imwrite(test_normal_mask_path+fileName+'.tiff',normal_mask.astype(np.uint8) )
        cv2.imwrite(test_image_path+fileName+'.tiff',image )
    else:
        cv2.imwrite(tumor_mask_path+fileName+'.tiff',tumor_mask )
        cv2.imwrite(normal_mask_path+fileName+'.tiff',normal_mask.astype(np.uint8) )
        cv2.imwrite(image_path+fileName+'.tiff',image )

  0%|          | 0/500 [00:00<?, ?it/s]